In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [255]:
import pandas as pd
from operator import add

## Calculate total trip amount in each hour each day

In [286]:
traindata = sc.textFile('2014_nyc_taxi_trips_neighbor_droppedna.csv').cache()

In [205]:
list(enumerate(traindata.first().split(',')))

[(0, ''),
 (1, 'fare_amount'),
 (2, 'pickup_datetime'),
 (3, 'passenger_count'),
 (4, 'weekday'),
 (5, 'weekend'),
 (6, 'day_night'),
 (7, 'distance'),
 (8, 'neighborhood')]

In [208]:
def HourAmountHoliday(pid, record):
    if pid == 0:
        next(record)
        
    import csv
    from operator import add
    readers = csv.reader(record)
    count = {}
    holidaylist = ['2014-01-01', '2014-01-20', '2014-02-17', '2014-04-16', '2014-05-11', '2014-05-26', '2014-06-15'\
                       , '2014-07-04', '2014-09-01', '2014-10-13', '2014-11-11', '2014-11-27', '2014-11-28', '2014-12-15'\
                       , '2014-12-26']
    for row in readers:
        datehr = row[2].split(':')[0].replace(' ', '-')
        date = row[2].split(' ')[0]
        holiday = None
        if date in holidaylist:
            holiday = 1
        else: holiday = 0
        count[datehr] = list(map(add, count.get(datehr, (0, holiday)), (1,0)))
    return count.items()

In [266]:
outcome4 = traindata.mapPartitionsWithIndex(HourAmountHoliday).map(lambda x: ((x[0], x[1][1]), x[1][0])).reduceByKey(add)\
.map(lambda x: (x[0][0], x[1], x[0][1])).collect()

In [268]:
outcome4[:3]

[('2014-07-21-14', 1095, 0),
 ('2014-05-09-09', 1221, 0),
 ('2014-03-03-23', 908, 0)]

In [269]:
outcome4df = pd.DataFrame(outcome4, columns=['datekey','tripPerHour', 'holiday'])

In [270]:
sum(outcome4df.tripPerHour)

7986341

In [273]:
outcome4df.sort_values(by='datekey').head()

,datekey,tripPerHour,holiday
1950,2014-01-01-00,1489,1
8381,2014-01-01-01,1434,1
2479,2014-01-01-02,1656,1
5174,2014-01-01-03,1155,1
6818,2014-01-01-04,900,1


## Merge with main dataset

In [237]:
traindf = pd.read_csv('2014_nyc_taxi_trips_neighbor_droppedna.csv')
traindf.head()

,Unnamed: 0,fare_amount,pickup_datetime,passenger_count,weekday,weekend,day_night,distance,neighborhood
0,0,12.5,2014-02-19 07:22:00,1,2,0,1,1.619554,"('Theater District', 'Gramercy')"
1,1,4.0,2014-12-06 20:36:22,1,5,1,0,0.247719,"('Murray Hill', 'Midtown')"
2,2,5.5,2014-02-19 16:03:00,1,2,0,1,0.541562,"('Midtown', 'Theater District')"
3,3,5.0,2014-07-16 10:57:00,6,2,0,1,0.267995,"('Chelsea', 'Chelsea')"
4,4,7.0,2014-05-01 09:12:00,6,3,0,1,0.901087,"('Upper East Side', 'Upper West Side')"


In [238]:
traindf.shape

(7986341, 9)

In [277]:
traincsv = traindf.copy()

In [278]:
traincsv.shape

(7986341, 9)

In [279]:
traincsv.head()

,Unnamed: 0,fare_amount,pickup_datetime,passenger_count,weekday,weekend,day_night,distance,neighborhood
0,0,12.5,2014-02-19 07:22:00,1,2,0,1,1.619554,"('Theater District', 'Gramercy')"
1,1,4.0,2014-12-06 20:36:22,1,5,1,0,0.247719,"('Murray Hill', 'Midtown')"
2,2,5.5,2014-02-19 16:03:00,1,2,0,1,0.541562,"('Midtown', 'Theater District')"
3,3,5.0,2014-07-16 10:57:00,6,2,0,1,0.267995,"('Chelsea', 'Chelsea')"
4,4,7.0,2014-05-01 09:12:00,6,3,0,1,0.901087,"('Upper East Side', 'Upper West Side')"


In [280]:
traincsv['datekey'] = traincsv['pickup_datetime']

In [281]:
traincsv['datekey'] = traincsv['datekey'].map(lambda x: x.split(':')[0].replace(' ', '-'))

In [282]:
traincsv.head()

,Unnamed: 0,fare_amount,pickup_datetime,passenger_count,weekday,weekend,day_night,distance,neighborhood,datekey
0,0,12.5,2014-02-19 07:22:00,1,2,0,1,1.619554,"('Theater District', 'Gramercy')",2014-02-19-07
1,1,4.0,2014-12-06 20:36:22,1,5,1,0,0.247719,"('Murray Hill', 'Midtown')",2014-12-06-20
2,2,5.5,2014-02-19 16:03:00,1,2,0,1,0.541562,"('Midtown', 'Theater District')",2014-02-19-16
3,3,5.0,2014-07-16 10:57:00,6,2,0,1,0.267995,"('Chelsea', 'Chelsea')",2014-07-16-10
4,4,7.0,2014-05-01 09:12:00,6,3,0,1,0.901087,"('Upper East Side', 'Upper West Side')",2014-05-01-09


In [314]:
trainfinal = pd.merge(traincsv, outcome4df, on='datekey')

In [315]:
trainfinal.head()

,Unnamed: 0,fare_amount,pickup_datetime,passenger_count,weekday,weekend,day_night,distance,neighborhood,datekey,tripPerHour,holiday
0,0,12.5,2014-02-19 07:22:00,1,2,0,1,1.619554,"('Theater District', 'Gramercy')",2014-02-19-07,1066,0
1,12023,10.5,2014-02-19 07:56:58,1,2,0,1,1.789184,"('Upper East Side', 'Murray Hill')",2014-02-19-07,1066,0
2,12053,8.0,2014-02-19 07:13:27,1,2,0,1,1.697914,"('Gramercy', 'Midtown')",2014-02-19-07,1066,0
3,49876,20.5,2014-02-19 07:33:52,2,2,0,1,1.720657,"('Upper East Side', 'Midtown')",2014-02-19-07,1066,0
4,55889,25.0,2014-02-19 07:09:00,1,2,0,1,4.554564,"('Upper East Side', 'Battery Park City')",2014-02-19-07,1066,0


In [316]:
trainfinal.shape

(7986341, 12)

In [317]:
trainfinal.drop(['Unnamed: 0', 'pickup_datetime'], axis=1, inplace=True)

In [320]:
trainfinal.head()

,fare_amount,passenger_count,weekday,weekend,day_night,distance,neighborhood,datekey,tripPerHour,holiday
0,12.5,1,2,0,1,1.619554,"('Theater District', 'Gramercy')",2014-02-19-07,1066,0
1,10.5,1,2,0,1,1.789184,"('Upper East Side', 'Murray Hill')",2014-02-19-07,1066,0
2,8.0,1,2,0,1,1.697914,"('Gramercy', 'Midtown')",2014-02-19-07,1066,0
3,20.5,2,2,0,1,1.720657,"('Upper East Side', 'Midtown')",2014-02-19-07,1066,0
4,25.0,1,2,0,1,4.554564,"('Upper East Side', 'Battery Park City')",2014-02-19-07,1066,0


In [323]:
trainfinal.to_csv('2014_nyc_taxi_trips_neighbor_final.csv')